# Dados

In [10]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.animation as animation
import numpy as np
import math

In [2]:
breast=pd.read_csv('dados/breast.csv',header=None)
breast=breast.rename(columns={breast.columns[30]:'y'})
breast

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,y
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0.0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0.0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0.0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0.0


# Normalizador

In [3]:
class Nomalizer:
    def __init__(self):
        self.y_max=0.0
        self.y_min=0.0
        self.x_max=[]
        self.x_min=[]

    def fit(self,x,y):
        # Pega os valores minimos de 
        self.y_max=y['y'].max().tolist()
        self.y_min=y['y'].min().tolist()

        # Pega os valores maximos e minimos do X
        self.x_max=x.max().tolist()
        self.x_min=x.min().tolist()
    
    def normalize(self,x,y):
        y_norm=y.map(self.y_norm_aux)
        x_norm=x.copy()
        norm_index=0
        for column in x.columns:
            x_column=[]
            for i in range(len(x[column])):
                x_column.append((x[column][i]-self.x_min[norm_index])/(self.x_max[norm_index]-self.x_min[norm_index]))
                
            norm_index+=1
            x_norm[column]=x_column    
        
        return x_norm,y_norm
    
    def desnormalize(self,x_norm,y_norm):
        y=y_norm.map(self.y_desnorm_aux)
        x=x_norm.copy()
        norm_index=0
        for column in x_norm.columns:
            x_column=[]
            for i in range(len(x_norm[column])):
                x_column.append((x_norm[column][i]*(self.x_max[norm_index]-self.x_min[norm_index]))+self.x_min[norm_index])
                
            norm_index+=1
            x[column]=x_column    
        
        return x,y
    
    def y_norm_aux(self,y):
        return (y-self.y_min)/(self.y_max-self.y_min)
    
    def y_desnorm_aux(self,y):
        return (y*(self.y_max-self.y_min))+self.y_min

# modelo

In [66]:
class LogisticClassifier:
  def __init__(self,opt='SGD'):
    self.w =[]
    self.losses=[]
    

  def fit(self, X, y, n, reg, lr=0.001, epochs=100):
    x_poli = self.polinomial_transfomation(X,n)# Transforma em polinomio
    df_bias = pd.DataFrame({'bias':[1]*len(x_poli)})
    X_bias=pd.concat([df_bias,x_poli], axis=1)
    self.w=np.ones(X_bias.shape[1])
    self.fit_SGD(X_bias,y,reg,lr,epochs)

  def polinomial_transfomation(self,x,n):
    df_columns = x.columns
    poli_columns={}
    self.n=n
    x_poli=x.copy()
    for column in df_columns:
      for i in range(2,n+1):
        poli_columns[f"{column}_{i}"] = x[column]**i
    poli_df = pd.DataFrame(poli_columns)
    x_poli = pd.concat([x,poli_df], axis=1)
    return x_poli


  def fit_SGD(self, X, y, reg, lr, epochs):
    for i in range(epochs):
      errors=[]
      indices = list(range(len(X)))
      random.shuffle(indices)

      for sample in indices:
        #Calculo dos erros
        y_pred=(X.iloc[sample]*self.w).sum()# esta certo
        y_sig=1 / (1 + np.exp(-y_pred))# esta certo
        error_np=y.iloc[sample]-y_sig# esta certo
        error=error_np.item()#esta certo
        errors.append(error)
        
        #ajuste dos Ws    
        w_arr = np.array(self.w)#esta certo
        inter = lr*error*X.iloc[sample]#esta certo
        w_novo = (reg*w_arr) + inter#esta certo
        self.w=w_novo
        
        #Calculo da loss
        self.loss_function(X, y, reg)
        
      
  def sigma(self,x):
    return 1/(1+np.exp(-x))
      
  def loss_function(self, x, y,reg):
    w=np.array(self.w)
    x_arr=x.to_numpy()
    y_arr=y.to_numpy()

    N = len(y_arr)#Numero de amostras
    p = np.dot(x_arr, w)#Produto interno para cada xi
    sigma = 1/(1+np.exp(-p))
    epsilon = 1e-15# sujeira
    sigma = np.clip(sigma, epsilon, 1 - epsilon)# Garante que não ocorra log(0)
    loss = -(1/N)*np.sum(y_arr*np.log(sigma)+(1-y_arr)*np.log(1-sigma)) #Calcula a perda e salva na lista
    self.losses.append(loss)
    

  def predict(self, X):
    y=0
    y_pred=[]# Lista para salvar as predições
    df_bias = pd.DataFrame({'bias':[1]*len(X)})
    x_poli = self.polinomial_transfomation(X,self.n)
    X_bias=pd.concat([df_bias,x_poli], axis=1)
    for sample in range(len(X_bias)):
      y_sigma=self.sigma((X_bias.iloc[sample]*self.w).sum())# Calcula a predição 
      y_pred.append(0 if y_sigma<=0.5 else 1)# Aplica a função degrau para definir a classe e salva na lista
    return y_pred

# Questão 1

In [65]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


In [47]:
x=breast.iloc[:, 0:30]  
y=breast[['y']]

In [70]:
len(x)

569

In [62]:
norm=Nomalizer()
norm.fit(x,y)
norm_Trx,norm_Try=norm.normalize(x,y)

In [49]:
norm_Trx

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,0.336364,0.132056,...,0.623266,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667
565,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,0.349495,0.113100,...,0.560655,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315
566,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,0.267677,0.137321,...,0.393099,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909
567,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,0.675253,0.425442,...,0.633582,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315


In [84]:
# Configurando o KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Para armazenar métricas de cada fold
mse_scores = []
acuracias =[]
revocacoes=[]
precisoes=[]
f1_scores=[]

# Loop pelos folds
for train_index, test_index in kf.split(x):
    # Dividindo os dados em treino e teste
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    # Normalizando os dados
    norm=Nomalizer()
    norm.fit(X_train,y_train)
    norm_Trx,norm_Try=norm.normalize(X_train,y_train)
    norm_Tsx,norm_Tsy=norm.normalize(X_test,y_test)


    # Treinando um modelo
    model = LogisticClassifier()
    model.fit(norm_Trx,norm_Try,n=1, reg=1, lr=0.001, epochs=50)

    # Fazendo previsões
    y_pred = model.predict(norm_Tsx)
    print("/////////////////////////////////////////////////")
    # Calculo da acuracia
    acetertos = np.sum(np.array(y_pred)==norm_Tsy.to_numpy())
    acuracia = acetertos / len(norm_Tsy)
    print("acuracia do modelo: ",acuracia)
    acuracias.append(acuracia)

    # Calculo da revocação
    TP = np.sum((np.array(y_pred) == 1) & (norm_Tsy.to_numpy() == 1))  # Previsões corretas para a classe positiva
    FN = np.sum((np.array(y_pred) == 0) & (norm_Tsy.to_numpy() == 1))  # Valores reais positivos, mas previstos como negativos
    revocacao = TP / (TP + FN)
    revocacoes.append(revocacao)
    print("revocação do modelo: ",revocacao)

    # Calculando a precisão
    VP = np.sum((norm_Tsy.to_numpy() == 1) & (np.array(y_pred) == 1))
    FP = np.sum((norm_Tsy.to_numpy() == 0) & (np.array(y_pred) == 1))
    precisao = VP / (VP + FP) 
    precisoes.append(precisao)
    print("precisao do modelo: ",precisao)

    # Calculando f1-score
    f1_score = 2 * (precisao * revocacao) / (precisao + revocacao)
    f1_scores.append(f1_score)
    print("f1-score do modelo: ",f1_score)




# Exibindo o resultado
print("/////////////////////////////////////////////////")
print("Acuracia médio:", np.mean(acuracias))
print("Acuracia em cada fold:", acuracias)

print("Revocação médio:", np.mean(revocacoes))
print("Revocações em cada fold:", revocacoes)

print("Precisão médio:", np.mean(precisoes))
print("Precisões em cada fold:", precisoes)

print("F1-score médio:", np.mean(f1_scores))
print("F1-scores em cada fold:", f1_scores)
print("/////////////////////////////////////////////////")


/////////////////////////////////////////////////
acuracia do modelo:  60.43859649122807
revocação do modelo:  0.6228070175438597
precisao do modelo:  0.6228070175438597
f1-score do modelo:  0.6228070175438597
/////////////////////////////////////////////////
acuracia do modelo:  63.666666666666664
revocação do modelo:  0.6666666666666666
precisao do modelo:  0.6754385964912281
f1-score do modelo:  0.6710239651416122
/////////////////////////////////////////////////
acuracia do modelo:  61.175438596491226
revocação do modelo:  0.6491228070175439
precisao do modelo:  0.6228070175438597
f1-score do modelo:  0.6356926799758015
/////////////////////////////////////////////////
acuracia do modelo:  61.175438596491226
revocação do modelo:  0.6491228070175439
precisao do modelo:  0.6228070175438597
f1-score do modelo:  0.6356926799758015
/////////////////////////////////////////////////
acuracia do modelo:  60.86725663716814
revocação do modelo:  0.7079646017699115
precisao do modelo:  0.5929

In [75]:
# Configurando o KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Para armazenar métricas de cada fold
mse_scores = []

# Loop pelos folds
for train_index, test_index in kf.split(x):
    # Dividindo os dados em treino e teste
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    # Normalizando os dados
    norm=Nomalizer()
    norm.fit(X_train,y_train)
    norm_Trx,norm_Try=norm.normalize(X_train,y_train)
    norm_Tsx,norm_Tsy=norm.normalize(X_test,y_test)


    # Treinando um modelo
    model = LogisticClassifier()
    model.fit(norm_Trx,norm_Try,n=2, reg=1, lr=0.001, epochs=50)

    # Fazendo previsões
    y_pred = model.predict(norm_Tsx)

    # Calculando a métrica (MSE neste caso)
    mse = mean_squared_error(norm_Tsy, y_pred)
    print(mse)
    mse_scores.append(mse)

# Exibindo o resultado
print("MSE médio:", np.mean(mse_scores))
print("MSE em cada fold:", mse_scores)

0.17543859649122806
0.21052631578947367
0.20175438596491227
0.15789473684210525
0.17699115044247787
MSE médio: 0.18452103710603943
MSE em cada fold: [0.17543859649122806, 0.21052631578947367, 0.20175438596491227, 0.15789473684210525, 0.17699115044247787]


In [74]:
# Configurando o KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Para armazenar métricas de cada fold
mse_scores = []

# Loop pelos folds
for train_index, test_index in kf.split(x):
    # Dividindo os dados em treino e teste
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    # Normalizando os dados
    norm=Nomalizer()
    norm.fit(X_train,y_train)
    norm_Trx,norm_Try=norm.normalize(X_train,y_train)
    norm_Tsx,norm_Tsy=norm.normalize(X_test,y_test)


    # Treinando um modelo
    model = LogisticClassifier()
    model.fit(norm_Trx,norm_Try,n=9, reg=1, lr=0.001, epochs=50)

    # Fazendo previsões
    y_pred = model.predict(norm_Tsx)

    # Calculando a métrica (MSE neste caso)
    mse = mean_squared_error(norm_Tsy, y_pred)
    print(mse)
    mse_scores.append(mse)

# Exibindo o resultado
print("MSE médio:", np.mean(mse_scores))
print("MSE em cada fold:", mse_scores)

0.3684210526315789
0.37719298245614036
0.41228070175438597
0.2894736842105263
0.34513274336283184
MSE médio: 0.35850023288309274
MSE em cada fold: [0.3684210526315789, 0.37719298245614036, 0.41228070175438597, 0.2894736842105263, 0.34513274336283184]


In [67]:
lc = LogisticClassifier()
lc.fit(norm_Trx,norm_Try,n=3, reg=1, lr=0.001, epochs=50)

In [68]:
lc.predict(norm_Trx)

[0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,


In [ ]:
lc2 = LogisticClassifier()
lc2.fit(norm_Trx,norm_Try,n=9, reg=1, lr=0.001, epochs=50)

In [51]:
lc2.predict(norm_Trx)

[0.2745198595142778,
 -2.5760181003767837,
 -3.461830586937145,
 22.243964297467457,
 -2.088091680949746,
 0.7370190412499196,
 -2.992796583121917,
 0.533314444762212,
 -0.028741162008451088,
 16.232338300876748,
 -0.5635042747848502,
 -1.784981560176302,
 5.252120979685005,
 -0.3983142215807758,
 0.5273344867983742,
 1.0041100502692861,
 -1.429521496152323,
 -1.26592682453211,
 -3.4225509856391056,
 -0.5859463172548072,
 0.29576779685645527,
 1.571947335272902,
 -0.8655454206016833,
 -2.0590255061538443,
 -1.9123344873795536,
 2.886686602464323,
 -2.495690519336417,
 -1.322931042172105,
 -1.5932103260092823,
 -1.796065484029672,
 -1.8514295719752945,
 1.760600167810931,
 -1.4340782896780606,
 -1.833309908255477,
 -1.794332960252547,
 -1.3798937650223566,
 -0.8962696984227025,
 0.9797814000255591,
 1.3483029566069402,
 -2.4186500111549227,
 -0.5701676608855488,
 1.3602710705784884,
 1.6626096718667185,
 -0.7983832053442157,
 -1.3379612297476833,
 -3.089962567300655,
 1.8597909076513788